## 全体尤度から計算される対数尤度比と，条件付き尤度から計算される対数尤度比を比較

以下のプログラムでは，同じデータに対する，次の3つの異なるモデルの対数尤度比を比較します．

** $r \times c$個のPoisson分布 **

$ Y_{ij} \sim Poi(\lambda_{ij}) $

** 1個の多項分布（$r \times c$項分布） = 全体の度数で条件付け**

$ Y_{ij} \sim Mult(n, p_{ij}) $

** r個の多項分布($c$項分布) = 行和で条件付け **

$ Y_{ij} \sim Mult(n_i, \pi_{j}) \ ({\rm for}\ i = 1,2, \cdots, r)$



In [63]:
#バージョンの表示
versioninfo()

Julia Version 1.3.0
Commit 46ce4d7933 (2019-11-26 06:09 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i3-5005U CPU @ 2.00GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.1 (ORCJIT, broadwell)


In [58]:
# 確率関数のパッケージをインポート
##import Pkg; Pkg.add("Distributions")

# 確率関数のパッケージは Distributions
using Distributions

#周辺度数を計算するためのサブ関数
function calc_marginal(y)
    c = sum(y, dims = 1)
    nc = length(c)
    r = sum(y, dims = 2)
    nr = length(r)
    n = sum(r)
    return c, nc, r, nr, n
end    

#r×c個のPoisson分布を仮定したときの尤度比
function calc_LR_poi(y)
    c, nc, r, nr, n = calc_marginal(y)
    #r×c個のPoisson分布：帰無仮説の下でのパラメータ推定値
    e0 = r * c / n
    lr = 0.0
    for i in 1:nr
        for j in 1:nc
            lr += log(pdf(Poisson(y[i,j]), y[i,j])) - log(pdf(Poisson(e0[i,j]), y[i,j]))        
        end
    end
    return lr
end

#1個の多項分布（r×c項分布）を仮定したときの尤度比
# 注意！：定数項（n!/(y11!y12! ...y_{rc}!)は面倒なので省略しました．
function calc_LR_mult(y)
    c, nc, r, nr, n = calc_marginal(y)
    #１個のr×c項分布：帰無仮説の下でのパラメータ推定値
    p0 = r * c / n^2
    p1 = y / n
    lr = 0.0
    for i in 1:nr
        for j in 1:nc
            lr += y[i, j] * log(p1[i, j]) - y[i, j] * log(p0[i, j])        
        end
    end
    return lr
end


#1個の多項分布（r×c項分布）を仮定したときの尤度比
# 注意！：定数項（n_i!/(y_{i1}!y_{i2}! ...y_{ic}!)は面倒なので省略しました．
function calc_LR_multiple_mult(y)
    c, nc, r, nr, n = calc_marginal(y)
    lr = 0.0
    p0 = c / n
    for i in 1:nr
        #r個のc項分布：帰無仮説の下でのパラメータ推定値
        p1 = y[i,:] / r[i]
        for j in 1:nc
            lr += y[i, j] * log(p1[j]) - y[i, j] * log(p0[j])        
        end
    end
    return lr
end



y = [3.0 4.0 8.0; 
    5.0 8.0 9.0]

print("r×c個のPoisson分布を仮定したときの対数尤度比", "\n")
print(calc_LR_poi(y), "\n\n")
print("1個の多項分布（r×c項分布）を仮定したときの対数尤度比", "\n")
print(calc_LR_mult(y), "\n\n")
print("r個の多項分布（c項分布）を仮定したときの対数尤度比", "\n")
print(calc_LR_multiple_mult(y), "\n\n")


r×c個のPoisson分布を仮定したときの対数尤度比
0.2955265331275825

1個の多項分布（r×c項分布）を仮定したときの対数尤度比
0.29552653312758714

r個の多項分布（c項分布）を仮定したときの対数尤度比
0.29552653312757915

